<a href="https://colab.research.google.com/github/NobinPegasus/sust-cse-result-pdf-to-csv-extractor/blob/main/result_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
import pdfplumber
import pandas as pd
import re
import numpy as np

In [ ]:
!python --version

Python 3.7.12


In [ ]:
! gdown --id 1bQFbunkSgdBLJxfzJfTPDGkiyZ8oWuZU
! unzip Results.zip

Downloading...
From: https://drive.google.com/uc?id=1bQFbunkSgdBLJxfzJfTPDGkiyZ8oWuZU
To: /content/Results.zip
100% 505k/505k [00:00<00:00, 45.0MB/s]
Archive:  Results.zip
replace second year first sem.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: second year first sem.pdf  
  inflating: second year second semester.pdf  
  inflating: 1st Year 1stSemester.pdf  
  inflating: 1st Year 2nd Semester Tabulation.pdf  


In [ ]:


def dataframe_generator(flat_list, counter):

    df = pd.DataFrame(flat_list,columns=flat_list[0])
    df.columns = df.columns.str.replace("\n"," ")
    for column in ["Reg No.", "Session"]:  
      df[column] = df[column].str.replace(" ", "")


    df = df.iloc[1: , :]
    if df['Reg No.'].iloc[0] == None:
      df = df[1:]

    names = df.columns.to_list()

    res = [str(i or '') for i in names]
    li = []

    for i in range(len(res)):

      a = re.findall("\d+\.\d+", res[i])
      li.append(a)

    sr = pd.Series(li)

    return df


In [ ]:
counter = 0

def flat_list_generator( range1, range2, filename):

    pdf = pdfplumber.open(filename)
    total = pdf.pages
    totalpages = len(total)
    global counter
    total_list = []
    for i in range(0 ,range1):
      t1 = total[i].extract_table()

      total_list.append(t1)


    globals()['flat_list_{}'.format(counter)] = [item for sublist in total_list for item in sublist]

    df = dataframe_generator(eval(f'flat_list_{counter}'), counter)
    df.drop_duplicates(subset=None, keep=False, inplace=True) 

    if(df['Reg No.'].duplicated().any()):
      df = df.groupby(['Reg No.'])
      df = df.first()

    counter += 1

    
    total_list = []
    if range2 is not None:
      for i in range(range1 ,range2):

        
        t1 = total[i].extract_table()

        total_list.append(t1)


      globals()['flat_list_{}'.format(counter)] = [item for sublist in total_list for item in sublist]

      df2 = dataframe_generator(eval(f'flat_list_{counter}'), counter)
      df2.drop_duplicates(subset=None, keep=False, inplace=True)

      counter += 1
    

    if(range2 is not None):
      data = pd.merge(df, df2, on=['Reg No.', 'Session', 'Course No. = Name Credit = Semester ='], sort= True)
      data.drop_duplicates(subset=None, keep=False, inplace=True)
      return data
    
    else:
      df.drop_duplicates(subset=None, keep=False, inplace=True)
      return df


    







In [ ]:
a = flat_list_generator(7, None, "1st Year 1stSemester.pdf")
b = flat_list_generator(8,16, '1st Year 2nd Semester Tabulation.pdf')
c = flat_list_generator(8,16, "second year first sem.pdf")
d = flat_list_generator(16,24, "second year second semester.pdf")



hello = d


In [ ]:



def custom_merge(first, second):

  first_frame = list(first.columns)
  second_frame = list(second.columns)

  identical_columnNames = []


  for i in first_frame:
    for j in second_frame:
      if(j==i):
        identical_columnNames.append(i)
  cols = [col for col in identical_columnNames if col is not None]
  if not 'Reg No.' in cols:
    cols = ['Reg No.'] + cols

  data = pd.merge(first, second, on=cols, how='outer', sort= True)

  return data

In [171]:
data = custom_merge(a,b)
data2 = custom_merge(data,c)
data3 = custom_merge(data2, d)


data3 = data3.replace('',np.nan)
data3 = data3.replace('0.00 F', np.nan)

data4=data3.groupby('Reg No.', dropna=True)
data4= data4.first()

data4 = data4.rename(columns={'Course No. = Name Credit = Semester =':'Name'})


data4.columns.values[15] = '1st sem CGPA'
data4.columns.values[16] = '1st sem letter grade'

data4.columns.values[29] = '2nd sem CGPA'
data4.columns.values[30] = '2nd sem letter grade'

data4.columns.values[40] = '3rd sem CGPA'
data4.columns.values[41] = '3rd sem letter grade'

data4.columns.values[49] = '4th sem CGPA'
data4.columns.values[50] = '4th sem letter grade'


data4 = data4.drop(['Total Credit', 'Total GPA','Letter Grade', 'Cumulative','Remarks','GC' ], axis = 1)

# data4['CSE250 1.00 4th Semester']
  # print('Kisui pai nai')
# data4.get_value[40,'CSE250 1.00 4th Semester']


# di.columns

# di = data4[(data4['CSE150 1.00 2nd Semester'] == None) | (data4['CSE150 1.00 2nd Semester'] <'3') | (data4['CSE250 1.00 4th Semester'] == None) | (data4['CSE250 1.00 4th Semester'] < '3') | (data4['CSE237 3.00 3rd Semester'] == None) | (data4['CSE237 3.00 3rd Semester'] < '3') | (data4['CSE238 2.00 3rd Semester'] < '3') | (data4['CSE238 2.00 3rd Semester'] == None)]

# di
# di = data4[(data4['Name']=='MD. EMTIAZ KABIR IFTU')].values[0]


# ji = di.iloc[]
# di
# ji = di[['CSE250 1.00 4th Semester']]


# print(type(ji))
# print(di.columns)

# di
hell = data4[['Name','CSE133 3.00 1st Semester','CSE134 2.00 1st Semester', 'CSE143 3.00 1st Semester','CSE144 1.00 1st Semester','CSE137 3.00 2nd Semester', 'CSE138 2.00 2nd Semester','CSE233 3.00 3rd Semester', 'CSE234 2.00 3rd Semester','CSE237 3.00 3rd Semester', 'CSE238 2.00 3rd Semester','CSE239 2.00 4th Semester', 'CSE240 1.50 4th Semester','CSE150 1.00 2nd Semester','CSE250 1.00 4th Semester']] 
# di = di[['Name', 'CSE150 1.00 2nd Semester','CSE250 1.00 4th Semester', 'CSE237 3.00 3rd Semester', 'CSE238 2.00 3rd Semester']]
haha = list(hell.columns.values)
print(haha[14])


# hell.columns
op = hell[((hell[haha[1]]<'3') & (hell[haha[2]]<'3')) | ((hell[haha[3]]<'3') & (hell[haha[4]]<'3')) | ((hell[haha[5]]<'3') & (hell[haha[6]]<'3')) | ((hell[haha[7]]<'3') & (hell[haha[8]]<'3')) | ((hell[haha[9]]<'3') & (hell[haha[10]]<'3')) | ((hell[haha[11]]<'3') & (hell[haha[12]]<'3')) | ((hell[haha[13]]<'3') & (hell[haha[14]]<'3'))]
op
# print(len(op))

# di = hell[(data4['CSE150 1.00 2nd Semester'] == None) | (data4['CSE150 1.00 2nd Semester'] <'3') | (data4['CSE250 1.00 4th Semester'] == None) | (data4['CSE250 1.00 4th Semester'] < '3') | (data4['CSE237 3.00 3rd Semester'] == None) | (data4['CSE237 3.00 3rd Semester'] < '3') | (data4['CSE238 2.00 3rd Semester'] < '3') | (data4['CSE238 2.00 3rd Semester'] == None)]

# hell.isnull().sum(1)>1

# di
# data4
# print(len(di))



CSE250 1.00 4th Semester


,Name,CSE133 3.00 1st Semester,CSE134 2.00 1st Semester,CSE143 3.00 1st Semester,CSE144 1.00 1st Semester,CSE137 3.00 2nd Semester,CSE138 2.00 2nd Semester,CSE233 3.00 3rd Semester,CSE234 2.00 3rd Semester,CSE237 3.00 3rd Semester,CSE238 2.00 3rd Semester,CSE239 2.00 4th Semester,CSE240 1.50 4th Semester,CSE150 1.00 2nd Semester,CSE250 1.00 4th Semester
Reg No.,,,,,,,,,,,,,,,
2016331063,SHAILA AKTER,None,None,2.75 B-,None,None,None,None,None,None,2.00 C-,2.25 C,2.00 C-,3.00 B,2.00 C-
2017331001,FAHIM UR RASHID,2.00 C-,4.00 A+,2.75 B-,2.50 C+,2.25 C,3.00 B,2.75 B-,3.50 A-,2.75 B-,3.00 B,2.75 B-,3.25 B+,3.00 B,3.50 A-
2017331003,TANJIL BHOIYAN,None,2.50 C+,2.00 C-,2.50 C+,None,2.00 C-,None,None,None,None,2.00 C-,3.00 B,2.75 B-,3.25 B+
2017331005,MD. MARUFUR RAHMAN,2.00 C-,4.00 A+,2.75 B-,3.75 A,2.50 C+,3.00 B,2.50 C+,4.00 A+,2.00 C-,2.00 C-,2.00 C-,2.50 C+,3.25 B+,3.00 B
2017331013,NAVID ISHRIYAQ AMIN,2.25 C,3.50 A-,2.75 B-,2.00 C-,2.50 C+,2.75 B-,3.00 B,3.75 A,2.50 C+,2.25 C,4.00 A+,4.00 A+,2.25 C,None
2017331020,MD.NAFIUL ISLAM PIASH,4.00 A+,4.00 A+,3.75 A,3.25 B+,3.25 B+,3.00 B,3.50 A-,3.25 B+,2.50 C+,2.00 C-,2.25 C,3.25 B+,3.25 B+,None
2017331021,ABHISHEK DEY,2.00 C-,3.25 B+,3.50 A-,4.00 A+,None,None,3.50 A-,3.50 A-,None,2.00 C-,3.25 B+,3.75 A,2.25 C,2.50 C+
2017331023,SANI TALUKDER,2.25 C,4.00 A+,4.00 A+,4.00 A+,2.75 B-,3.00 B,3.50 A-,3.75 A,2.75 B-,2.25 C,4.00 A+,4.00 A+,3.75 A,3.25 B+
2017331024,SHAKIL AHMED,3.00 B,4.00 A+,3.50 A-,3.75 A,3.00 B,3.50 A-,3.00 B,3.75 A,2.75 B-,2.75 B-,2.25 C,3.75 A,2.75 B-,3.25 B+


In [110]:
# def nans(df): return df[df.isnull().any(axis=1)]

# df.loc[df.isnull().sum(1)>1].index

In [163]:
# nans(data4)

# data4
# data4.isnull()
# data4.isnull().any(axis=1)
# data4[data4.isna().any(axis=1)]

# data4.loc[data4.isnull().sum(1)>1]
# data4

# data4 = hell.drop(['CSE134 3.00 1st Semester'], axis = 1)
lp=hell.loc[hell.isnull().sum(1)>1]
# print(len(lp))
# print(len(data4))
lp



28


,Name,CSE133 3.00 1st Semester,CSE134 2.00 1st Semester,CSE143 3.00 1st Semester,CSE144 1.00 1st Semester,CSE137 3.00 2nd Semester,CSE138 2.00 2nd Semester,CSE233 3.00 3rd Semester,CSE234 2.00 3rd Semester,CSE237 3.00 3rd Semester,CSE238 2.00 3rd Semester,CSE239 2.00 4th Semester,CSE240 1.50 4th Semester,CSE150 1.00 2nd Semester,CSE250 1.00 4th Semester
Reg No.,,,,,,,,,,,,,,,
2016331063,SHAILA AKTER,None,None,2.75 B-,None,None,None,None,None,None,2.00 C-,2.25 C,2.00 C-,3.00 B,2.00 C-
2016331077,RUHUL.QUDUS.MIRAZ,None,None,None,None,None,None,None,None,None,None,None,2.75 B-,None,None
2017331003,TANJIL BHOIYAN,None,2.50 C+,2.00 C-,2.50 C+,None,2.00 C-,None,None,None,None,2.00 C-,3.00 B,2.75 B-,3.25 B+
2017331007,OBHI DATTA,2.00 C-,None,None,4.00 A+,None,None,None,3.50 A-,None,None,None,2.00 C-,None,3.00 B
2017331009,FAHIM AL AMIN,None,2.50 C+,None,3.75 A,None,None,2.25 C,3.00 B,None,None,None,2.50 C+,2.50 C+,None
2017331010,M. EBRAHIM SAZIN,2.25 C,3.00 B,3.50 A-,3.25 B+,2.75 B-,3.50 A-,3.25 B+,None,2.00 C-,None,3.25 B+,3.75 A,2.50 C+,None
2017331016,LAZU ISLAM,2.00 C-,None,3.75 A,4.00 A+,2.25 C,None,None,None,3.00 B,2.00 C-,3.25 B+,3.25 B+,2.25 C,None
2017331018,IFTEKHAR AHMED RANA,None,None,None,3.00 B,None,None,None,None,None,None,None,None,None,None
2017331021,ABHISHEK DEY,2.00 C-,3.25 B+,3.50 A-,4.00 A+,None,None,3.50 A-,3.50 A-,None,2.00 C-,3.25 B+,3.75 A,2.25 C,2.50 C+


In [55]:
data4.to_csv('all_results.csv')

In [ ]:
pd.set_option('max_rows', 99999)


In [ ]:


# data1 = df.loc[df['CSE237 3.00 3rd Semester'] <= '3' & df['CSE238 2.00 3rd Semester'] <= '3']

# dji = data1.loc[data1['Course No. = Name Credit = Semester ='] == 'RAFAT ASHRAF JOY']
# hul = dji[['CSE150 1.00 2nd Semester']]




# di = den[['Course No. = Name Credit = Semester =', 'CSE150 1.00 2nd Semester','CSE250 1.00 4th Semester','CSE237 3.00 3rd Semester', 'CSE238 2.00 3rd Semester']]
# di = (di['CSE150 1.00 2nd Semester'] <'3') or (di['CSE250 1.00 4th Semester']) or ((di['CSE237 3.00 3rd Semester'] < '3') & (di['CSE238 2.00 3rd Semester'] < '3'))

# di = di[((di['CSE150 1.00 2nd Semester'] != '') & (di['CSE150 1.00 2nd Semester'] <'3')) | ((di['CSE250 1.00 4th Semester'] != '') & (di['CSE250 1.00 4th Semester'] < '3')) | ((di['CSE237 3.00 3rd Semester'] < '3') & (di['CSE238 2.00 3rd Semester'] < '3'))]

# di


# data2 = df.loc[((df['CSE237 3.00 3rd Semester'] < '3') & (df['CSE238 2.00 3rd Semester'] < '3'))]
